In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl

In [49]:
bird_strike = pd.read_csv('birdstrikes.csv', delimiter=',', low_memory=False)

In [50]:
with open('insert_is_bird.pkl', 'rb') as picklefile:
    bird_or_not = pkl.load(picklefile)

### Merge bird_strike and bird_or_not to add 'is_bird' column into bird_strike (main df)

In [51]:
df = pd.merge(bird_strike, bird_or_not, on='Species Name')

### Categorize all 'unknown' as 'Bird' in is_bird column

In [52]:
# Original shape
df.shape

(174104, 67)

In [53]:
# only want 'UNKNOWN SMALL BIRD', 'UNKNOWN MEDIUM BIRD', 'UNKNOWN LARGE BIRD'
df[df['is_bird'] == 'unknown']['Species Name'].value_counts()

UNKNOWN MEDIUM BIRD           38531
UNKNOWN SMALL BIRD            32981
UNKNOWN BIRD                   6400
UNKNOWN LARGE BIRD             2789
UNKNOWN BIRD OR BAT              70
UNKNOWN TERRESTRIAL MAMMAL       25
Name: Species Name, dtype: int64

In [54]:
# only want 'UNKNOWN SMALL BIRD', 'UNKNOWN MEDIUM BIRD', 'UNKNOWN LARGE BIRD'
df = df[df['Species Name'] != 'UNKNOWN TERRESTRIAL MAMMAL']
df = df[df['Species Name'] != 'UNKNOWN BIRD OR BAT']
df = df[df['Species Name'] != 'UNKNOWN BIRD']


In [55]:
# After subsetting Dataframe shape
# (174104 - 25 - 70 - 6400) rows = 167609 rows
df.shape

(167609, 67)

In [56]:
df.reset_index(inplace=True)

We only care about subsetting 'birds' out. So below is a list of is_bird categories that we may want to change to 'Bird':

In [57]:
df['is_bird'].value_counts()

Bird                     82173
unknown                  74301
Animal                    4817
None                      2674
Clothing company          1574
Toiletries                1041
multiple                   316
Rodent                     249
Reptile                    200
Television programme        95
none                        80
Fictional character         47
Baseball team                8
Football franchise           8
Snakes                       7
Software                     7
Basketball team              5
Rock band                    2
Company                      2
Manufacturing company        1
Chess                        1
Football team                1
Name: is_bird, dtype: int64

In [58]:
df[df['is_bird'] == 'None']['Species Name'].unique()

array(['BLACKBIRD', 'CHIMNEY SWIFT', 'OLD WORLD VULTURES',
       'NORTHERN HARRIER', 'YELLOW-BILLED CUCKOO', 'FOX SPARROW',
       'EASTERN TOWHEE', 'MONK PARAKEET', 'BOBOLINK', 'CHICKEN TURTLE',
       'ISLAND TURTLE-DOVE', 'WHITE-THROATED SWIFT', 'SWINE (PIGS)',
       'COMMON POORWILL', 'GREEN-TAILED TOWHEE', "VAUX'S SWIFT",
       'LARK SPARROW', 'SAGE THRASHER', 'NORTH AMERICAN PORCUPINE',
       'VESPER SPARROW', 'DIAMONDBACK TERRAPIN', 'REDHEAD',
       'HAIRY WOODPECKER', 'SILVER-HAIRED BAT', "HARRIS'S HAWK",
       'FORK-TAILED STORM-PETREL', 'WRENTIT', 'SKUNK',
       'SWALLOW-TAILED KITE', 'PERCHING BIRDS (Z)',
       'PROTHONOTARY WARBLER', 'ISLAND CANARY', 'BLACK-BILLED CUCKOO',
       'TENNESSEE WARBLER', 'NORTHERN HAWK-OWL',
       'ALLIGATOR SNAPPING TURTLE', 'FLORIDA BONNETED BAT',
       'BAND-RUMPED STORM-PETREL', 'NORTHERN WATER SNAKE',
       'SPOTTED TOWHEE', 'PHILIPPINE DRONGO-CUCKOO'], dtype=object)

In [59]:
for_sure_birds_to_change = ['UNKNOWN SMALL BIRD', 'UNKNOWN MEDIUM BIRD', 'UNKNOWN LARGE BIRD',
                            'BLACKBIRD','CHIMNEY SWIFT','OLD WORLD VULTURES','NORTHERN HARRIER',
                            'WILD TURKEY','YELLOW-BILLED CUCKOO','FOX SPARROW','EASTERN TOWHEE','MONK PARAKEET',
                            'BOBOLINK','ISLAND TURTLE-DOVE','WHITE-THROATED SWIFT', 'COMMON POORWILL',
                            'GREEN-TAILED TOWHEE',"VAUX'S SWIFT", 'LARK SPARROW', 'SAGE THRASHER', 'VESPER SPARROW', 
                            'REDHEAD','HAIRY WOODPECKER', "HARRIS'S HAWK",'FORK-TAILED STORM-PETREL', 'WRENTIT',
                            'SWALLOW-TAILED KITE', 'PERCHING BIRDS (Z)','PROTHONOTARY WARBLER', 'ISLAND CANARY',
                            'BLACK-BILLED CUCKOO','TENNESSEE WARBLER','NORTHERN HAWK-OWL','BAND-RUMPED STORM-PETREL', 
                            'SPOTTED TOWHEE','PHILIPPINE DRONGO-CUCKOO']

In [60]:
def bird_categorize_None(x):
    if x in for_sure_birds_to_change:
        return 'Bird'
    else:
        return 'Not Bird'

is_bird_None = df[df['is_bird'] == 'None']
categorize_None_bird = [bird_categorize_None(x) for x in is_bird_None['Species Name']]

In [61]:
df['is_bird'][is_bird_None.index] = is_bird_None['is_bird']

In [62]:
# All UNKNOWN's are birds
df[df['is_bird'] == 'unknown']['Species Name'].unique()

array(['UNKNOWN MEDIUM BIRD', 'UNKNOWN LARGE BIRD', 'UNKNOWN SMALL BIRD'],
      dtype=object)

In [63]:
def unknown_to_bird(x):
    if x == 'unknown':
        return 'Bird'
    else:
        return x

is_bird_unknown = [unknown_to_bird(x) for x in df['is_bird']]
df['is_bird'] = is_bird_unknown

In [64]:
df[df['is_bird'] =='multiple']['Species Name'].unique()

array(['DUCKS, GEESE, SWANS', 'THRASHERS, MOCKINGBIRDS, CATBIRDS',
       'HERONS, EGRETS, BITTERNS', 'GULLS,TERNS, KITTIWAKES',
       'CROWS, RAVENS, JAYS,MAGPIES',
       'RAPTORS: HAWKS, EAGLES, VULTURES, KITES, OSPREY, F',
       'BLACKBIRDS, MEADOWLARKS, ORIOLES', 'PIGEONS, DOVES',
       'WOODPECKERS, PICULETS', 'PLOVERS, LAPWINGS',
       'SCOPS, SCREECH OWLS', 'CUCKOOS, ROADRUNNERS, ANIS',
       'KITES, EAGLES, HAWKS', 'LAGOMORPHS (RABBITS, HARES)',
       'WAXBILLS, MANNIKINS, PARROTFINCHES', 'CROWS, RAVENS'],
      dtype=object)

In [65]:
def multiple_to_bird(m):
    if m == 'multiple':
        return 'Bird'
    else:
        return m

is_bird_multiple = [multiple_to_bird(m) for m in df['is_bird']]
df['is_bird'] = is_bird_multiple

In [66]:
df['is_bird'][166845] = 'Animal'
df[df['Species Name'] == 'LAGOMORPHS (RABBITS, HARES)']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,index,Record ID,Incident Year,Incident Month,Incident Day,Operator ID,Operator,Aircraft,Aircraft Type,Aircraft Make,...,Fuselage Damage,Landing Gear Strike,Landing Gear Damage,Tail Strike,Tail Damage,Lights Strike,Lights Damage,Other Strike,Other Damage,is_bird
166845,173340,258786,2008,7,1,BUS,BUSINESS,PA-31 NAVAJO,A,729,...,0,0,0,0,0,0,0,0,0,Animal


In [67]:
df['is_bird'].value_counts()

Bird                     156789
Animal                     4818
None                       2674
Clothing company           1574
Toiletries                 1041
Rodent                      249
Reptile                     200
Television programme         95
none                         80
Fictional character          47
Baseball team                 8
Football franchise            8
Snakes                        7
Software                      7
Basketball team               5
Rock band                     2
Company                       2
Manufacturing company         1
Chess                         1
Football team                 1
Name: is_bird, dtype: int64

In [68]:
## Remaining List
# 'Television programme', 'Clothing company','Toiletries', 'Baseball team', 'Fictional character',
# 'Software', 'Company', 'Basketball team','Rock band', 'Football franchise','Manufacturing company', 
# 'Chess','Football team'

In [69]:
df[df['is_bird'] == 'Football team']['Species Name'].unique()

array(['RAVENS'], dtype=object)

In [70]:
#bird_or_not[bird_or_not['is_bird'] == 'Television programme']['Species Name'].unique() # all Merlin birds
#bird_or_not[bird_or_not['is_bird'] == 'Clothing company']['Species Name'].unique() # all Canada Goose (birds)
#bird_or_not[bird_or_not['is_bird'] == 'Toiletries']['Species Name'].unique() # all Dove (birds)
#bird_or_not[bird_or_not['is_bird'] == 'Baseball team']['Species Name'].unique() # all Orioles (birds)
#bird_or_not[bird_or_not['is_bird'] == 'Fictional character']['Species Name'].unique() # all Sora (birds)
#bird_or_not[bird_or_not['is_bird'] == 'Software']['Species Name'].unique() # all Rails (birds)
#bird_or_not[bird_or_not['is_bird'] == 'Basketball team']['Species Name'].unique() # Pelicans (birds)
#bird_or_not[bird_or_not['is_bird'] == 'Rock band']['Species Name'].unique() # Swans (birds)
#bird_or_not[bird_or_not['is_bird'] == 'Football franchise']['Species Name'].unique() # Eagles (birds)
#bird_or_not[bird_or_not['is_bird'] == 'Manufacturing company']['Species Name'].unique() # Redwing (birds)
#bird_or_not[bird_or_not['is_bird'] == 'Football team']['Species Name'].unique() # Ravens (birds)
#bird_or_not[bird_or_not['is_bird'] == 'Chess']['Species Name'].unique() # Rook (birds)

In [71]:
remain_birds = ['Television programme', 'Clothing company','Toiletries', 'Baseball team',
                'Fictional character', 'Software', 'Basketball team', 'Chess',
                'Rock band', 'Football franchise','Manufacturing company','Football team']

def remain_to_bird(r):
    if r in remain_birds:
        return 'Bird'
    else:
        return r

is_bird_remain = [remain_to_bird(r) for r in df['is_bird']]
df['is_bird'] = is_bird_remain


In [72]:
# Sanity check
df['is_bird'].value_counts()

Bird       159579
Animal       4818
None         2674
Rodent        249
Reptile       200
none           80
Snakes          7
Company         2
Name: is_bird, dtype: int64

In [73]:
df_filled = df[df['is_bird'] != 'none']
df_filled['is_bird'].value_counts()

Bird       159579
Animal       4818
None         2674
Rodent        249
Reptile       200
Snakes          7
Company         2
Name: is_bird, dtype: int64

In [74]:
def is_bird(b):
    if b == 'Bird':
        return 1
    else:
        return 0

In [75]:
is_it_bird = [is_bird(b) for b in df_filled['is_bird']]
df_filled['is_bird'] = is_it_bird

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [76]:
df_filled['is_bird'].value_counts()

1    159579
0      7950
Name: is_bird, dtype: int64

### Unstack all cells with multiple Species Names into individual rows

In [77]:
b = df_filled['Species Name'].str.split(',', expand=True).stack()
j = b.index.get_level_values(0)
df_complete = df_filled.loc[j].copy().reset_index()
df_complete['Species Name'] = b.values

In [78]:
df_complete.drop(columns=['level_0', 'index'], inplace=True)
df_complete.head()

,Record ID,Incident Year,Incident Month,Incident Day,Operator ID,Operator,Aircraft,Aircraft Type,Aircraft Make,Aircraft Model,...,Fuselage Damage,Landing Gear Strike,Landing Gear Damage,Tail Strike,Tail Damage,Lights Strike,Lights Damage,Other Strike,Other Damage,is_bird
0,127128,1990,1,1,DAL,DELTA AIR LINES,B-757-200,A,148,26,...,0,0,0,1,1,0,0,0,0,1
1,107155,1990,1,8,USA,1US AIRWAYS,B-737-200,A,148,13,...,0,0,0,0,0,0,0,0,0,1
2,100363,1990,2,22,NHK,FAA,CV-580,A,279,14,...,0,0,0,0,0,0,0,0,0,1
3,100098,1990,3,3,SWA,SOUTHWEST AIRLINES,B-737-200,A,148,13,...,0,0,0,0,0,0,0,0,0,1
4,102449,1990,3,18,ANA,ALL NIPPON AIRWAYS,B-747,A,148,96,...,0,0,0,0,0,0,0,0,0,1


In [79]:
with open('bird_df_clean.pkl', 'wb') as picklefile:
    pkl.dump(df_complete, picklefile)

### Clean Bird Sizes

In [230]:
with open('bird_sizes_df.pkl', 'rb') as picklefile:
    size_df = pkl.load(picklefile)

with open('bird_df_clean.pkl', 'rb') as file:
    clean_df = pkl.load(file)

In [231]:
size_df.head(3)

,Species Name,Species Size,Species Length,Species Weight
0,GULL,Size: </span>\nLarge (16 - 32 in)\n\n,Length Range: </span>\n51 cm (20 in)\n\n,Weight: </span>\n567 g (20 oz)\n\n
1,HOUSE SPARROW,Size: </span>\nSmall (5 - 9 in)\n\n,Length Range: </span>\n14-17 cm (5.5-6.5 in),Weight: </span>\n39 g (1.37 oz)\n\n
2,BARN OWL,Size: </span>\nLarge (16 - 32 in)\n\n,Length Range: </span>\n36-51 cm (14-20 in)\n\n,Weight: </span>\n510 g (18 oz)\n\n


In [232]:
size_df['Species Size'].unique()

array(['Size: </span>\nLarge (16 - 32 in)\n\n         ',
       'Size: </span>\nSmall (5 - 9 in)\n\n           ', 'None',
       'Size: </span>\nMedium (9 - 16 in)\n\n         ',
       'Size: </span>\nVery Large (32 - 72 in)\n\n    ', '',
       'Size: </span>\nVery Small (3 - 5 in)\n\n      '], dtype=object)

In [233]:
def define_size(s):
    if 'Very Small' in s:
        return 1
    elif 'Small' in s:
        return 2
    elif 'Medium' in s:
        return 3
    elif 'Large' in s:
        return 4
    elif 'Very Large' in s:
        return 5
    elif '' in s:
        return 'None'
    else:
        return 'None'

In [234]:
sp_size = [define_size(s) for s in size_df['Species Size']]
size_df['Species Size'] = sp_size

In [235]:
size_df['Species Size'][38] = 2  # 'UNKNOWN SMALL BIRD' size
size_df['Species Size'][3] = 3   # 'UNKNOWN MEDIUM BIRD' size
size_df['Species Size'][21] = 4  # 'UNKNOWN LARGE BIRD' size

In [236]:
just_size_df = size_df[['Species Name', 'Species Size']]
just_size_df['Species Size'].unique()

array([4, 2, 3, 'None', 1], dtype=object)

In [193]:
df2 = pd.merge(just_size_df, clean_df, on='Species Name')
df2.head(3)

,Species Name,Species Size,Record ID,Incident Year,Incident Month,Incident Day,Operator ID,Operator,Aircraft,Aircraft Type,...,Fuselage Damage,Landing Gear Strike,Landing Gear Damage,Tail Strike,Tail Damage,Lights Strike,Lights Damage,Other Strike,Other Damage,is_bird
0,GULL,4,127128,1990,1,1,DAL,DELTA AIR LINES,B-757-200,A,...,0,0,0,1,1,0,0,0,0,1
1,GULL,4,107155,1990,1,8,USA,1US AIRWAYS,B-737-200,A,...,0,0,0,0,0,0,0,0,0,1
2,GULL,4,100363,1990,2,22,NHK,FAA,CV-580,A,...,0,0,0,0,0,0,0,0,0,1


In [194]:
yes_bird_df = df2[df2['is_bird'] == 1]
missing_size_species = yes_bird_df[yes_bird_df['Species Size'] == 'None']['Species Name'].unique()
missing_size_species

array(['MUNIAS', 'GEESE', 'HAWAIIAN DUCK', 'DUCKS', ' GEESE', ' SWANS',
       'THRASHERS', ' MOCKINGBIRDS', ' CATBIRDS', 'HERONS', ' EGRETS',
       ' BITTERNS', "TOWNSEND'S SHEARWATER", 'BLACK VULTURE', 'SHOREBIRD',
       'TERNS', 'STARLINGS', 'OWLS', 'NEW WORLD VULTURES',
       'GREAT FRIGATEBIRD', 'GULLS', ' KITTIWAKES', 'CROWS', ' RAVENS',
       ' JAYS', 'MAGPIES', 'ORIOLES', 'THRUSHES', 'UPLAND SANDPIPER',
       'RAPTORS: HAWKS', ' EAGLES', ' VULTURES', ' KITES', ' OSPREY',
       ' F', 'PLOVERS', 'SCALY-BREASTED MUNIA', 'CHESTNUT MUNIA',
       'EASTERN SCREECH-OWL', 'PARROTS', 'RAILS',
       'AMERICAN GOLDEN-PLOVER', 'BLACKBIRDS', ' MEADOWLARKS', ' ORIOLES',
       'LARK BUNTING', 'PTARMIGANS', 'PIGEONS', ' DOVES', 'SWIFTS',
       'EURASIAN BLACKBIRD', 'GREBES', 'PELICANS', 'COMMON MOORHEN',
       'IBISES', 'WOOD WARBLER', 'CORMORANTS', 'SWANS',
       'AUSTRALIAN PELICAN', 'EAGLES', 'WOODPECKERS', ' PICULETS',
       'YELLOW BITTERN', 'QUAILS', 'EURASIAN THICK-KNEE',
  

In [237]:
# 3-5 in
very_small = ['MUNIAS','SCALY-BREASTED MUNIA','CHESTNUT MUNIA','WOOD WARBLER',' PICULETS','PERCHING BIRDS',
              'CHICKADEES','YELLOW-RUMPED WARBLER','GRASSHOPPER SPARROW',"WILSON'S WARBLER",'WHITE-THROATED MUNIA',
              'SAGE SPARROW','PIPITS','LESSER GOLDFINCH','HUMMINGBIRDS','WAXBILLS',' MANNIKINS',' PARROTFINCHES',
              "NELSON'S SHARP-TAILED SPARROW","LE CONTE'S SPARROW"]

# 5-9 in
small = [' MOCKINGBIRDS',' CATBIRDS','SHOREBIRD','STARLINGS','ORIOLES','THRUSHES','UPLAND SANDPIPER','PLOVERS',
         'EASTERN SCREECH-OWL',' ORIOLES','LARK BUNTING','SWIFTS','QUAILS','LEAST SANDPIPER','TYRANT FLYCATCHERS',
         'BUFF-BREASTED SANDPIPER','TANAGERS','PECTORAL SANDPIPER','LITTLE TERN','VIREOS','SCOPS',' SCREECH OWLS',
         'LARKS','NIGHTJARS','BOHEMIAN WAXWING','MOUNTAIN BLUEBIRD','TROPICAL MOCKINGBIRD','PIPING PLOVER',
         'LITTLE OWL','WHITE-CRESTED ELAENIA','TOWHEES','BLACK-FACED BUNTING','RED PHALAROPE','SONG THRUSH','MOCKINGBIRDS',
         'MEADOW PIPIT']

#9-16 in
medium = ['HAWAIIAN DUCK','DUCKS','THRASHERS',"TOWNSEND'S SHEARWATER",'TERNS','OWLS',' KITTIWAKES',' JAYS',
          'RAPTORS: HAWKS','RAILS','AMERICAN GOLDEN-PLOVER','BLACKBIRDS',' MEADOWLARKS','PTARMIGANS',' DOVES',
          'EURASIAN BLACKBIRD','GREBES','COMMON MOORHEN','WOODPECKERS','YELLOW BITTERN','EURASIAN THICK-KNEE',
          ' LAPWINGS','BONIN PETREL','SOUTHERN LAPWING','EURASIAN KESTREL','FAIRY TERN','HOODED CROW','CUCKOOS',
          ' ANIS','COMMON WOOD-PIGEON','RED-LEGGED PARTRIDGE','BLACK DRONGO','PUFFINS','NACUNDA NIGHTHAWK',
          'SHEARWATERS','SOUTH AMERICAN SNIPE','OLIVE-THROATED PARAKEET','PICAZURO PIGEON']

#16-32 in
large = ['GEESE',' GEESE','BLACK VULTURE','NEW WORLD VULTURES','GULLS','CROWS',' RAVENS','MAGPIES',' EAGLES',
         ' VULTURES',' KITES',' OSPREY','PIGEONS','IBISES','EAGLES','TROPICBIRDS','BLACK KITE','CARRION CROW',
         'EURASIAN BUZZARD',' ROADRUNNERS','PHILIPPINE DUCK','KITES',' HAWKS','ROOK','LOONS','DOUBLE-STRIPED THICK-KNEE',
         'INTERMEDIATE EGRET','GREAT CRESTED GREBE','RAVENS','YELLOW-HEADED CARCARA']

#32-72 in
very_large = [' SWANS','HERONS',' EGRETS',' BITTERNS','GREAT FRIGATEBIRD','PARROTS','PELICANS','CORMORANTS','SWANS',
              'AUSTRALIAN PELICAN','LAPPET-FACED VULTURE','WHITE STORK','WHITE-BELLIED SEA-EAGLE','GRAY HERON',
              "BRANDT'S CORMORANT",'CRANES']

# not_a_bird = [' F']

In [196]:
#missing_size_df = pd.DataFrame(columns=['Species Name', 'Species Size'])
#missing_size_df['Species Name'] = missing_size_species

In [238]:
for i in range(0, len(just_size_df)):
    if just_size_df['Species Name'][i] in very_small:
        just_size_df['Species Size'][i] = 1
    
    elif just_size_df['Species Name'][i] in small:
        just_size_df['Species Size'][i] = 2
        
    elif just_size_df['Species Name'][i] in medium:
        just_size_df['Species Size'][i] = 3
    
    elif just_size_df['Species Name'][i] in large:
        just_size_df['Species Size'][i] = 4
    
    elif just_size_df['Species Name'][i] in very_large:
        just_size_df['Species Size'][i] = 5
    
    else:
        just_size_df['Species Size'][i] = just_size_df['Species Size'][i]

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [239]:
df3 = pd.merge(just_size_df, clean_df, on='Species Name')

In [248]:
df3 = df3[df3['Species Name'] != ' F']
final_df = df3[df3['is_bird'] == 1]
final_df['Species Size'].value_counts()

3    68010
2    61964
4    27591
1     2417
5      246
Name: Species Size, dtype: int64

In [249]:
with open('final_df.pkl', 'wb') as picklefile:
    pkl.dump(final_df ,picklefile)